# Distill-BERT pretrained on SQUaD

Load the pretrained model

In [1]:
from transformers import DistilBertTokenizer, TFDistilBertForQuestionAnswering,pipeline
import tensorflow as tf

model_name = "deepset/roberta-base-squad2"
model = pipeline('question-answering', model=model_name, tokenizer=model_name)

/home/kxlu/anaconda3/envs/omnixai/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-18 15:13:37.541605: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 15:13:37.604965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 15:13:39.042760: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could n

Example answering

In [2]:
question, text = "Who is Mizuki?", "Mizuki is a jellyfish."

outputs = model({"context":text,"question":question})

print(outputs['answer'])

jellyfish


In [12]:
from omnixai.data.text import Text
# Build the prediction function, e.g., the context and question are seperated by "[SEP]".
def _predict(x: Text):
    x = x.split(sep="[SEP]")
    inputs = [{"context": y[0], "question": y[1]} for y in x]
    outputs = model(inputs)
    if isinstance(outputs, dict):
        outputs = [outputs]
    return [output["answer"] for output in outputs]
def preprocess(x: Text):
    x = x.split(sep="[SEP]")
    inputs = [{"context": y[0], "question": y[1]} for y in x]
    return inputs

In [17]:
from omnixai.explainers.nlp.specific.ig import IntegratedGradientText
# Initialize a NLPExplainer
explainer = IntegratedGradientText(
    mode="qa",
    model=model.model,
    preprocess_function=preprocess,
    embedding_layer=model.model.get_input_embeddings(),
    id2token=model.tokenizer,
    predict_function = _predict
)

x = Text([
    "The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks. [SEP] "
    "What can people do with model coversion?",
    "Electric vehicles emit much less harmful pollutants than conventional vehicles and ultimately, create a cleaner environment for human beings. [SEP] "
    "what is the result of using eletric vehicles?"
])
# Generates explanations
local_explanations = explainer.explain(x)

AttributeError: 'dict' object has no attribute 'to'

In [4]:
from omnixai.explainers.nlp.counterfactual.polyjuice import Polyjuice
# Initialize a NLPExplainer
explainer = Polyjuice(predict_function=_predict, mode="qa")
x = Text([
    "The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks. [SEP] "
    "What can people do with model coversion?",
    "Electric vehicles emit much less harmful pollutants than conventional vehicles and ultimately, create a cleaner environment for human beings. [SEP] "
    "what is the result of using eletric vehicles?"
])
# Generates explanations
local_explanations = explainer.explain(x)

INFO:polyjuice.polyjuice_wrapper:Setup Polyjuice.
INFO:polyjuice.polyjuice_wrapper:Setup SpaCy processor.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: `early_stopping` must be a boolean or 'never', but is None.